In [1]:
import cv2
import json
from os import listdir
from os.path import isfile, join
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
json_files_root = "../extractedbdhandspeakskeletons/skeleton_subtitle_jsons/"
raw_videos_root = "../extractedbdhandspeakskeletons/reduced_framerate_raw_videos/"
cropped_video_root = "../extractedbdhandspeakskeletons/cropped_videos/"
skeleton_csv_file_root = "../extractedbdhandspeakskeletons/skeleton_csv/"

In [3]:
raw_video_file_names = [f for f in listdir(raw_videos_root) if isfile(join(raw_videos_root, f))]

In [4]:
def FrameCapture(path):
    cap = cv2.VideoCapture(path)
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    image_array = []
    fc = 0
    ret = True
    while (fc < frameCount  and ret):
        ret, image = cap.read()
        image_array.append(image)
        fc += 1

    cap.release()
    return image_array, frameWidth, frameHeight

def CreateVideo(path, list_of_frames, width, height):
    out = cv2.VideoWriter(path,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (width, height))
    for frame in list_of_frames:
        out.write(frame)
    out.release()


In [16]:
for raw_video_file_name in raw_video_file_names:
    # print(f"processing raw_video_file: {raw_video_file_name}")
    json_file_name = raw_video_file_name.split('.')[0] + '.json'
    json_file_object = open(f"{json_files_root}/{json_file_name}", encoding='utf-8')
    json_content = json.load(json_file_object)

    if isfile(f"{cropped_video_root}/{raw_video_file_name.split('.')[0]}.avi"):
        continue

    # print(f"reading frames...")
    frames, width, height = FrameCapture(f"{raw_videos_root}/{raw_video_file_name}")
    # print(f"width: {width}, height: {height}")
    # print(f"frame count: {len(frames)}")

    # print(f"reading skeleton csv")
    skeleton_dataframe = pd.read_csv(f"{skeleton_csv_file_root}/{raw_video_file_name.split('.')[0].strip()}.csv")
    # print(f"dataframe size: {skeleton_dataframe.shape[0]}")

    cropped_frames = []

    final_frame_height = int((7*height) / 8)
    final_frame_width = width//2

    sample_frame_index = skeleton_dataframe.shape[0]//2
    sample_center_x = int(skeleton_dataframe.loc[sample_frame_index, 'pose_1_x'])
    frame_left = width //2
    frame_right = width

    # print(f"final height {final_frame_height} width {final_frame_width}")
    # print(f"final left {frame_left} right {frame_right}")
    # print(f"shape: ", frames[sample_frame_index].shape )

    for index, row in skeleton_dataframe.iterrows():
        if index >= len(frames):
            break

        frame = frames[index][:final_frame_height,frame_left - (width //6):frame_right - (width //6)]
        cropped_frames.append(frame)

    # print(f"creating video...")
    CreateVideo(f"{cropped_video_root}/{raw_video_file_name.split('.')[0]}.avi", cropped_frames, final_frame_width, final_frame_height)
    # break
    print("DONE")
    

DONE
DONE


KeyboardInterrupt: 